## CAPSTONE PROJECT DATA CONSOLIDATION
## AUTHOR: Simon Lee
## DATE CREATED: June 17th 2022
## LAST DATE ADJUSTED: June 17th 2022

Importing Packages

In [3]:
import pandas as pd
import csv as csv
import os as os
from lxml import html
import requests
from bs4 import BeautifulSoup


In [ ]:
cols = ['PlayerID', 'Full Name', 'Born', 'Age', 'Batting Style', 'Bowling Style', 'Playing Role']
a = []
for i in info:
    for x in cols:
        try:
            a.append([i][0][0][x])
        except:
            a.append('')
player_details = pd.DataFrame(data = data, columns = cols)

Function to Scrape ESPN Cricinfo for player roles

In [267]:
apiPlStatsUrl = "https://www.espncricinfo.com/player/kemar-roach-{pid}"
def getData(pid):
    meta = {'PlayerID':pid}
    page = requests.get(apiPlStatsUrl.format(pid=pid))
    html = BeautifulSoup(page.text, 'html.parser')
    cols = ['PlayerID', 'Full Name', 'Born', 'Age', 'Batting Style', 'Bowling Style', 'Playing Role']
    a = []

    for i in range (0, len(html.findAll('p', 'ds-text-tight-m ds-font-regular ds-uppercase ds-text-ui-typo-mid'))):
        meta[html.findAll('p', 'ds-text-tight-m ds-font-regular ds-uppercase ds-text-ui-typo-mid')[i].text] = html.findAll('span', 'ds-text-title-s ds-font-bold ds-text-ui-typo')[i].text

    
    for x in cols:
        try:
            a.append(meta[x])
        except:
            a.append('')
        
    return [a]



[[670035, 'Brandon Alexander King', 'December 16, 1994, Kingston, Jamaica', '27y 210d', 'Right hand Bat', '', 'Top order Batter']]


In [268]:
print(getData(670035))

[[670035, 'Brandon Alexander King', 'December 16, 1994, Kingston, Jamaica', '27y 210d', 'Right hand Bat', '', 'Top order Batter']]


Reading in all ball by ball data and people list

In [5]:
ball = pd.read_csv(r"/Users/simonlee/Desktop/Capstone/Ball by Ball Data.csv")
people = pd.read_csv(r"/Users/simonlee/Desktop/Capstone/Data/people.csv")

In [6]:
df1 = ball.groupby(['Striker ID', 'Striker']).size().reset_index().rename(columns = {0:'count', 'Striker': 'Player', 'Striker ID': 'ID'}).drop("count", axis = 1)
df2 = ball.groupby(['Bowler ID', 'Bowler']).size().reset_index().rename(columns = {0:'count', 'Bowler': 'Player', 'Bowler ID':'ID'}).drop("count", axis = 1)
all_players =pd.concat([df1, df2])
unique_players = pd.DataFrame(data = all_players['ID'].unique(), columns= ['ID'])


In [7]:
##Getting all the people with cricinfo IDs
ids = unique_players.merge(people, how = 'left', left_on='ID', right_on='identifier').drop(['unique_name', 'key_bigbash',
       'key_cricbuzz', 'key_crichq', 'key_cricinfo_2',
       'key_cricingif', 'key_cricketarchive', 'key_cricketarchive_2',
       'key_opta', 'key_opta_2', 'key_pulse', 'key_pulse_2'], axis = 1)
int_id = []
no_id = []
for i in ids['key_cricinfo']:
    if i > 0:
        int_id.append(int(i))
    else:
        int_id.append(0)
    
ids['key_cricinfo'] = int_id

id = ids[ids['key_cricinfo'] > 0]

In [10]:
ids[ids["ID"] == '2498e163']

,ID,identifier,name,key_cricinfo
564,2498e163,2498e163,JR Hopes,5702


In [ ]:
##Getting all player roles using the getData function
info = []
count = 0
for i in id['key_cricinfo']:
    count = count + 1
    if count%50 ==0:
        info.append(getData(i))
        print(count)
    else:
        info.append(getData(i))


In [306]:

info2 = []
for i in info:
    info2.append(i[0])


roles = pd.DataFrame(data = info2, columns = ['PlayerID', 'Full Name', 'Born', 'Age', 'Batting Style', 'Bowling Style', 'Playing Role'])
roles.to_csv(r"/Users/simonlee/Desktop/Capstone/Data/roles.csv")

In [ ]:
#Getting info for players that were left out once the code shut down

leftovers = []
count = 0
for i in id['key_cricinfo']:
    print(i)
    if i in list(roles['PlayerID']) or i == 35320:
        print("already there")
    else:
        leftovers.append(getData(i)[0])
        count = count +1 
        print (count)

In [336]:
#Joinging The inital run of the web scrape and the players that were not included in the first run
df6 = pd.DataFrame(data = leftovers, columns = ['PlayerID', 'Full Name', 'Born', 'Age', 'Batting Style', 'Bowling Style', 'Playing Role'])
df7 = pd.concat([df6, roles])

In [2]:
df7.to_csv(r"/Users/simonlee/Desktop/Capstone/Data/roles.csv", index = 0)

NameError: name 'df7' is not defined